In [ ]:
%matplotlib inline


# Maxwell 3D: Magnetostatic Mutual Inductance Analysis


In [61]:
from pyaedt import Maxwell3d
from pyaedt import generate_unique_project_name
import os
import numpy as np

## Launch AEDT
Launch AEDT 2022 R2 in graphical mode.



In [3]:
m3d = Maxwell3d(projectname=generate_unique_project_name(), specified_version="2022.2", new_desktop_session=True, non_graphical=False)

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\v222\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2022.2 Started with process ID 27580.
pyaedt INFO: pyaedt v0.6.28
pyaedt INFO: Python version 3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 06:57:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: Project Project_QJQ has been created.
pyaedt INFO: No design is present. Inserting a new design.
pyaedt INFO: Added design 'Maxwell 3D_CFY' of type Maxwell 3D.
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded
pyaedt INFO: aedt file load time 0.0020012855529785156


## Set up Maxwell solution
Set up the Maxwell solution to DC.



In [4]:
m3d.solution_type = m3d.SOLUTIONS.Maxwell3d.Magnetostatic

In [97]:
def create_geometry(modeler):
    modeler.model_units = 'um'
    modeler.delete() # delete all objects to start fresh
    length = 50
    width = 5
    thickness = 5
    material = 'copper'
    distance = 20
    air = modeler.create_box([-50, -50, 0], [100, 100, length], name="Air", matname='air')
    wire1 = modeler.create_box([-distance/2 - width/2, 0, 0], [width, thickness, length], name="Wire1", matname=material)
    wire2 = modeler.create_box([distance/2 + width/2, 0, 0], [width, thickness, length], name="Wire2", matname=material)

    return air, wire1, wire2


In [79]:
air, wire1, wire2 = create_geometry(m3d.modeler)
# m3d.modeler.unite([wire1,wire2])

c1top = m3d.assign_current(wire1.bottom_face_z, 5, name='c1top')
c1bottom = m3d.assign_current(wire1.top_face_z, 5, swap_direction=True, name='c1bottom')

c2top = m3d.assign_current(wire2.bottom_face_z, 5, name='c2top')
c2bottom = m3d.assign_current(wire2.top_face_z, 5, swap_direction=True, name='c2bottom')
# wire1.rotate(cs_axis='z', angle=45)

for setup in m3d.setup_names:
    m3d.delete_setup(setup)
m3d.assign_matrix(['c1top', 'c2top'], matrix_name='current_matrix')
solSetup = m3d.create_setup(setupname='Sol1', props={'MinimumPasses': 3})
solSetup.props['MinimumPasses'] = 3
# solSetup.available_properties

pyaedt INFO: Deleted 3 Objects: Air,Wire1,Wire2.


In [ ]:
m3d.analyse_nominal()

In [101]:
m3d.create_output_variable('kappa','current_matrix.CplCoef(c2top,c1top)')
m3d.create_output_variable('L1','current_matrix.L(c1top,c1top)')
m3d.create_output_variable('L2','current_matrix.L(c2top,c2top)')
m3d.create_output_variable('M','current_matrix.L(c1top,c2top)')

True

In [60]:
def print_calc(L1, L2, M):
    print('k=',M/np.sqrt(L1*L2))
    print('L1=',L1,', L2=',L2)
    print('M=',M)
    h = 6.62607015e-34  # m2 kg / s
    e = 1.60217663e-19 # C
    fluxquant = h / (2 * e)
    I_needed = fluxquant / M # A
    print('current that achieves one flux quantum: ', I_needed * 1000, 'mA')

    jcNb = 2e5 # A/cm^2
    Nb_thickness = 100e-9 * 100 # cm
    track_width = I_needed / jcNb / Nb_thickness #cm
    print('Required min trackwidth: ', track_width*1e4, 'um')

In [108]:
kappa = m3d.get_output_variable('kappa')
L1 = m3d.get_output_variable('L1')
L2 = m3d.get_output_variable('L2')
M = m3d.get_output_variable('M')



k= 0.606471766866992
L1= 3.2418427969896603e-11 , L2= 3.25098720413542e-11
M= 1.96885708870901e-11
current that achieves one flux quantum:  0.10502711778742563 mA
Required min trackwidth:  0.5251355889371282 um


In [ ]:
m3d.save_project()
m3d.release_desktop(close_projects=True, close_desktop=True)

# HFSS

In [110]:
from pyaedt import Hfss, Hfss3dLayout

In [111]:
hfss = Hfss(new_desktop_session=True, specified_version="2022.2", non_graphical=False)

hfss.solution_type = "Terminal"

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\v222\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2022.2 Started with process ID 25736.
pyaedt INFO: pyaedt v0.6.28
pyaedt INFO: Python version 3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 06:57:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: Project Project4 has been created.
pyaedt INFO: No design is present. Inserting a new design.
pyaedt INFO: Added design 'HFSS_E6Z' of type HFSS.
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


In [112]:
air, wire1, wire2 = create_geometry(hfss.modeler)


pyaedt WARNING: No objects to delete


In [109]:
hfss.save_project()
hfss.release_desktop(close_projects=True, close_desktop=True)

pyaedt INFO: Project Project2 Saved correctly
pyaedt INFO: Project files removed from handlers.
pyaedt INFO: Project files removed from handlers.


True

# Q3D

In [89]:
import os

from pyaedt import Q3d
from pyaedt import generate_unique_project_name

In [90]:
q = Q3d(projectname=generate_unique_project_name(), specified_version="2022.2", non_graphical=False, new_desktop_session=True)

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\v222\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2022.2 Started with process ID 15624.
pyaedt INFO: pyaedt v0.6.28
pyaedt INFO: Python version 3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 06:57:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: Project Project_KVG has been created.
pyaedt INFO: No design is present. Inserting a new design.
pyaedt INFO: Added design 'Q3D Extractor_YB9' of type Q3D Extractor.
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded
pyaedt INFO: aedt file load time 0.0020020008087158203


In [98]:
air, wire1, wire2 = create_geometry(q.modeler)

pyaedt INFO: Deleted 3 Objects: Air,Wire1,Wire2.


In [99]:
source1 = q.assign_source_to_objectface(object_name='Wire1', axisdir=q.AxisDir.ZPos, source_name='source1')
sink1 = q.assign_sink_to_objectface(object_name='Wire1', axisdir=q.AxisDir.ZNeg, sink_name='sink1')

source2 = q.assign_source_to_objectface(object_name='Wire2', axisdir=q.AxisDir.ZPos, source_name='source2')
sink2 = q.assign_sink_to_objectface(object_name='Wire2', axisdir=q.AxisDir.ZNeg, sink_name='sink2')

q.auto_identify_nets()

pyaedt INFO: 2 Nets have been identified: Wire2, Wire1


True

In [100]:
solSetup = q.create_setup(setupname='Sol1')
solSetup.props['MinimumPasses'] = 3
#solSetup.available_properties

In [101]:
q.analyse_nominal()

pyaedt INFO: Solving design setup Sol1_2
pyaedt INFO: Design setup Sol1_2 solved correctly in 0.0h 0.0m 20.0s


In [102]:
def retrieve_q3d(q, i,j):
    res = q.post.get_solution_data(expressions=f'DCL(Wire{i}:source{i},Wire{j}:source{j})')
    if list(res.units_data.values())[0]!='nH':
        print('Data does not have the units nH which I have assumed')
    return res.data_real()[0] * 1e-9

In [103]:
L1 = retrieve_q3d(q,1,1)
L2 = retrieve_q3d(q,2,2)
M = retrieve_q3d(q,1,2)
print_calc(L1,L2,M)

pyaedt INFO: Solution Data Correctly Loaded.
pyaedt INFO: Solution Data Correctly Loaded.
pyaedt INFO: Solution Data Correctly Loaded.
k= 0.28729158423407924
L1= 2.871296554683524e-11 , L2= 2.877096236419887e-11
M= 8.257320156112963e-12
current that achieves one flux quantum:  0.250424328296591 mA
Required min trackwidth:  1.2521216414829552 um


In [104]:
q.save_project()
q.release_desktop(close_projects=True, close_desktop=True)

pyaedt INFO: Project Project_KVG Saved correctly
pyaedt INFO: Project files removed from handlers.
pyaedt INFO: Project files removed from handlers.


True